In [5]:
import pandas as pd
import numpy as np

def read_excel_to_vectors(file_path, sheet_name):
    """
    从Excel文件的指定sheet中读取数据并返回向量列表和对应的标题。
    
    参数:
    file_path: Excel文件的路径。
    sheet_name: 要读取数据的sheet名称。
    
    返回:
    vectors: 包含向量的列表，每个向量是NumPy数组。
    row_labels: 行标题的列表。
    col_labels: 列标题的列表。
    """
    df = pd.read_excel(file_path, sheet_name=sheet_name, index_col=0)  # 使用第一列作为行索引
    #将DataFrame中的字符串转换为数值，遇到非数值的数据则转为NaN
    df = df.apply(pd.to_numeric, errors='coerce')
    vectors = [np.array(row) for _, row in df.iterrows()]
    row_labels = df.index.tolist()
    col_labels = df.columns.tolist()
    return vectors, row_labels, col_labels

def cosine_similarity_matrix(vectors):
    """
    计算给定向量列表中每两个向量之间的余弦相似度，并返回相似度矩阵。
    
    参数:
    vectors: 一个包含向量的列表，每个向量是NumPy数组。
    
    返回:
    similarity_matrix: 余弦相似度矩阵，其中元素 (i, j) 表示第 i 个和第 j 个向量之间的余弦相似度。
    """
    num_vectors = len(vectors)
    similarity_matrix = np.zeros((num_vectors, num_vectors))

    for i in range(num_vectors):
        for j in range(num_vectors):
            if i == j:
                similarity_matrix[i, j] = 1.0  # 向量与自身的相似度为1
            else:
                similarity_matrix[i, j] = cosine_similarity(vectors[i], vectors[j])

    return similarity_matrix

def cosine_similarity(vector1, vector2):
    """
    计算两个向量之间的余弦相似度。
    
    参数:
    vector1: 第一个向量，是NumPy数组。
    vector2: 第二个向量，是NumPy数组。
    
    返回:
    similarity: 余弦相似度值。
    """
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    
    similarity = dot_product / (norm_vector1 * norm_vector2)
    
    return similarity

# 示例用法：
excel_file_path = 'mediadata.xlsx'  # 替换成你的Excel文件路径
output_excel_path = 'media_cosnet1.xlsx'  # 替换成你想要保存的新Excel文件路径

# 读取每个sheet表的数据，并生成余弦相似度矩阵
with pd.ExcelFile(excel_file_path) as xls:
    writer = pd.ExcelWriter(output_excel_path, engine='xlsxwriter')
    
    for sheet_name in xls.sheet_names:
        vectors, row_labels, col_labels = read_excel_to_vectors(excel_file_path, sheet_name)
        similarity_matrix = cosine_similarity_matrix(vectors)
        
        # 创建一个DataFrame来保存余弦相似度矩阵，使用行列标题
        similarity_df = pd.DataFrame(similarity_matrix, index=row_labels, columns=row_labels)
        
        # 将DataFrame写入新的Excel文件，每个sheet对应一个余弦相似度矩阵
        similarity_df.to_excel(writer, sheet_name=sheet_name, index=True)

    writer.save()

print(f"余弦相似度矩阵已保存到 {output_excel_path}")


余弦相似度矩阵已保存到 media_cosnet1.xlsx
